<a href="https://colab.research.google.com/github/Devinarukula57/FMML_M1L2/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-(Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
Yes, averaging validation accuracy across multiple splits (e.g., using cross-validation) generally provides more consistent and reliable results. Here's why:

### 1. **Reduces Overfitting to a Specific Split:**
   In a single train-test split, the model’s performance can vary significantly depending on how the data is divided. The split might be biased (e.g., having easier or harder samples in one of the subsets). Averaging across multiple splits mitigates this bias, leading to a more accurate estimate of model performance.

### 2. **Improves Robustness:**
   Cross-validation (like k-fold cross-validation) allows the model to be evaluated on different subsets of data. By averaging the performance metrics, you capture a broader view of the model’s generalization ability, leading to more consistent and stable results.

### 3. **Captures Variability in the Data:**
   In real-world datasets, there may be variations or imbalances (e.g., class distributions, outliers). Multiple splits ensure that the model is tested against different scenarios, leading to a more reliable estimate of how it would perform on unseen data.

### 4. **Provides a Confidence Interval:**
   Besides giving an average accuracy, multiple splits allow you to compute variance or standard deviation, offering a confidence interval around your performance metric. This helps in understanding the stability of the model.

### Conclusion:
Averaging validation accuracy across multiple splits is a best practice for obtaining consistent and generalizable performance estimates, especially in cases of limited data.
2. Does it give more accurate estimate of test accuracy?
Yes, averaging validation accuracy across multiple splits generally provides a more accurate estimate of test accuracy. Here’s why:

### 1. **Reduces the Risk of a Biased Estimate:**
   If you rely on a single train-test split, the specific data in that split might not be fully representative of the overall dataset. For example, it might contain an unusually easy or difficult subset, leading to either an overly optimistic or pessimistic estimate. By averaging across multiple splits (like in cross-validation), you smooth out these biases, leading to a more accurate estimate of how the model will perform on unseen test data.

### 2. **More Representative of the Full Dataset:**
   In k-fold cross-validation, the model is trained and validated on different portions of the data, ensuring that every sample is both in the training and validation sets at some point. This comprehensive use of the data leads to a better estimate of the model’s performance on truly unseen data.

### 3. **Generalization Ability:**
   Averaging results over multiple splits accounts for variations in the dataset, providing a better estimate of how well the model generalizes to new data. The test accuracy is more likely to be close to the average validation accuracy obtained from multiple splits.

### 4. **Reduces Variance:**
   Single train-test splits can be highly variable, especially with small datasets. Multiple splits reduce this variance, leading to a more stable and hence more accurate prediction of test accuracy.

### Conclusion:
While the validation accuracy across multiple splits may not perfectly match the test accuracy, it generally provides a much more reliable estimate than using a single split. The averaging process captures the variability within the dataset and results in a more accurate reflection of real-world performance.
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
Yes, generally, increasing the number of iterations (or folds) in cross-validation can lead to a better estimate of the model’s performance, but there are important considerations:

### 1. **Improved Estimate with More Iterations:**
   With more iterations (e.g., more folds in k-fold cross-validation or more random splits in repeated random sampling), you increase the variety of data subsets the model is trained and tested on. This leads to a more comprehensive assessment of the model’s performance, reducing the influence of any single split’s bias or noise. As a result, the estimate of test accuracy becomes more reliable.

### 2. **Diminishing Returns:**
   While increasing iterations generally improves the estimate, there are diminishing returns. Beyond a certain point, the gain in estimate accuracy becomes marginal. For example, moving from 5-fold to 10-fold cross-validation provides a noticeable improvement, but increasing from 10-fold to 20-fold may only slightly improve the estimate while significantly increasing computation time.

### 3. **Stability of the Estimate:**
   More iterations can lead to a more stable (lower variance) estimate. With few iterations, the estimate can be overly sensitive to the specific splits, resulting in higher variability. As the number of iterations increases, the estimate stabilizes, leading to a more accurate average.

### 4. **Computational Cost:**
   Higher iterations also come with a cost in terms of time and computational resources. While more iterations give a better estimate, the trade-off between accuracy and computational cost should be considered, especially in large datasets or complex models.

### 5. **Convergence of the Estimate:**
   As the number of iterations increases, the estimate tends to converge towards a consistent value. This indicates that the model's performance is well-captured and that further iterations provide little additional benefit in improving the estimate’s accuracy.

### Conclusion:
Increasing the number of iterations generally leads to a better and more accurate estimate of test accuracy. However, after a certain point, the improvement is marginal, and the cost in terms of computation time might outweigh the benefits. Therefore, while higher iterations can lead to a better estimate, the optimal number should balance accuracy with computational efficiency.
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
Increasing the number of iterations can help to some extent when working with a small training or validation dataset, but it has limitations. Here’s how it works and where it falls short:

### 1. **How Increasing Iterations Helps:**
   - **Maximizes Data Utilization:** In small datasets, each sample is crucial. With techniques like k-fold cross-validation (especially with high k values), every data point is used in both training and validation multiple times. This maximizes the utilization of limited data.
   - **Reduces Variance:** By averaging the results over many iterations, the model’s performance estimate becomes more stable. This is particularly valuable when there’s limited data, where a single split might give misleading results.
   - **Better Generalization Estimate:** By iterating through multiple different splits, the model is evaluated under various conditions, which can improve the estimate of how well the model would generalize to unseen data.

### 2. **Limitations of Increasing Iterations with Very Small Datasets:**
   - **Insufficient Training Data Per Split:** When the dataset is very small, even with many iterations, the training sets in each split might be too small for the model to learn effectively. This can lead to overfitting, where the model performs well on the limited data but fails to generalize to new data.
   - **High Variance Despite More Iterations:** With very limited data, each split might produce highly variable results even if you average across many iterations. The model’s performance could be too sensitive to slight differences in the small datasets, leading to unreliable estimates.
   - **Diminished Returns:** For very small datasets, increasing iterations (e.g., moving from 5-fold to 10-fold or even leave-one-out cross-validation) may offer little improvement in performance estimation. The improvement can be marginal while significantly increasing computational effort.

### 3. **When Increasing Iterations Isn’t Enough:**
   - **Data Augmentation:** If possible, generating additional synthetic samples through data augmentation can help improve model training with small datasets.
   - **Transfer Learning:** For small datasets, transfer learning can be very effective. Pre-training a model on a larger dataset and fine-tuning it on your small dataset can boost performance.
   - **Simpler Models:** Using simpler models or regularization can help avoid overfitting when data is very limited.

### Conclusion:
While increasing iterations does help deal with small datasets by maximizing data usage and stabilizing performance estimates, it has limits. When the dataset is extremely small, increasing iterations alone won’t overcome the challenges of insufficient training data or high variance. In such cases, you might need to combine this approach with techniques like data augmentation, transfer learning, or simpler models to achieve better results.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.